In [ ]:
# GUÍA COMPLETA DE HERRAMIENTAS LANGCHAIN
# =====================================
# Esta guía cubre todas las principales herramientas disponibles en LangChain

import os
from typing import Optional, Dict, Any, List
from langchain.tools import Tool, BaseTool, StructuredTool
from langchain.pydantic_v1 import BaseModel, Field
from langchain.callbacks.manager import CallbackManagerForToolRun

# ===============================================
# 1. HERRAMIENTAS DE BÚSQUEDA WEB
# ===============================================

# 1.1 DuckDuckGo Search
from langchain_community.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults

# Búsqueda simple
ddg_search = DuckDuckGoSearchRun()

# Búsqueda con resultados estructurados
ddg_results = DuckDuckGoSearchResults(
    num_results=5,  # Número de resultados
    region="es-es"  # Región específica
)

# Uso:
# resultado = ddg_search.run("inteligencia artificial 2024")

# 1.2 Google Search (requiere API key)
from langchain_google_community import GoogleSearchAPIWrapper

os.environ["GOOGLE_CSE_ID"] = "tu-cse-id"
os.environ["GOOGLE_API_KEY"] = "tu-google-api-key"

google_search = GoogleSearchAPIWrapper(
    google_cse_id=os.environ["GOOGLE_CSE_ID"],
    google_api_key=os.environ["GOOGLE_API_KEY"],
    k=5  # Número de resultados
)

google_tool = Tool(
    name="Google_Search",
    description="Búsqueda en Google para información actual",
    func=google_search.run
)

# 1.3 Bing Search (requiere API key)
from langchain_community.utilities import BingSearchAPIWrapper

os.environ["BING_SUBSCRIPTION_KEY"] = "tu-bing-key"

bing_search = BingSearchAPIWrapper(
    bing_subscription_key=os.environ["BING_SUBSCRIPTION_KEY"],
    k=5
)

bing_tool = Tool(
    name="Bing_Search",
    description="Búsqueda en Bing",
    func=bing_search.run
)

# 1.4 Serper Search (alternativa más económica)
from langchain_community.utilities import SerperAPIWrapper

os.environ["SERPER_API_KEY"] = "tu-serper-key"

serper_search = SerperAPIWrapper()
serper_tool = Tool(
    name="Serper_Search",
    description="Búsqueda web con Serper",
    func=serper_search.run
)

# ===============================================
# 2. HERRAMIENTAS DE ARCHIVOS Y DOCUMENTOS
# ===============================================

# 2.1 File Management Tools
from langchain_community.tools.file_management import (
    ReadFileTool,
    WriteFileTool,
    ListDirectoryTool,
    MoveFileTool,
    DeleteFileTool,
    CopyFileTool
)

# Leer archivos
read_tool = ReadFileTool(base_dir="./documents")

# Escribir archivos
write_tool = WriteFileTool(base_dir="./output")

# Listar directorio
list_tool = ListDirectoryTool(base_dir="./")

# Mover archivos
move_tool = MoveFileTool(base_dir_src="./temp", base_dir_dst="./archive")

# Eliminar archivos
delete_tool = DeleteFileTool(base_dir="./temp")

# Copiar archivos
copy_tool = CopyFileTool(base_dir_src="./source", base_dir_dst="./backup")

# 2.2 CSV Tools
from langchain_community.agent_toolkits.csv.base import create_csv_agent
from langchain_openai import ChatOpenAI

# Crear agente para CSV
def crear_csv_tool(archivo_csv: str):
    llm = ChatOpenAI(temperature=0)
    agente_csv = create_csv_agent(
        llm,
        archivo_csv,
        verbose=True,
        agent_type="openai-functions"
    )
    
    return Tool(
        name="CSV_Analyzer",
        description=f"Analiza y consulta datos del archivo CSV: {archivo_csv}",
        func=lambda query: agente_csv.run(query)
    )

# 2.3 PDF Tools
from langchain_community.tools import PyPDFium2DocumentLoaderTool

pdf_tool = PyPDFium2DocumentLoaderTool()

# ===============================================
# 3. HERRAMIENTAS DE BASES DE DATOS
# ===============================================

# 3.1 SQL Database
from langchain_community.agent_toolkits.sql.base import create_sql_agent
from langchain_community.utilities import SQLDatabase

# Conectar a base de datos
db = SQLDatabase.from_uri("sqlite:///example.db")

def crear_sql_tool():
    llm = ChatOpenAI(temperature=0)
    sql_agent = create_sql_agent(
        llm=llm,
        db=db,
        agent_type="openai-functions",
        verbose=True
    )
    
    return Tool(
        name="SQL_Query",
        description="Ejecuta consultas SQL en la base de datos",
        func=lambda query: sql_agent.run(query)
    )

# 3.2 Vector Database (para búsqueda semántica)
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.tools.vectorstore.tool import VectorStoreQATool

# Crear herramienta de vector store
def crear_vectorstore_tool(documentos: List[str], nombre: str):
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_texts(documentos, embeddings)
    
    return VectorStoreQATool(
        name=f"Vector_Search_{nombre}",
        description=f"Búsqueda semántica en {nombre}",
        vectorstore=vectorstore,
        llm=ChatOpenAI(temperature=0)
    )

# ===============================================
# 4. HERRAMIENTAS DE APIS Y WEB
# ===============================================

# 4.1 Requests Tool (para APIs REST)
from langchain_community.tools import RequestsGetTool, RequestsPostTool

# GET requests
requests_get = RequestsGetTool(
    allowed_schemas=["https", "http"],
    requests_wrapper=None  # Puedes añadir headers aquí
)

# POST requests  
requests_post = RequestsPostTool(
    allowed_schemas=["https", "http"]
)

# 4.2 OpenWeatherMap (clima)
from langchain_community.utilities import OpenWeatherMapAPIWrapper

os.environ["OPENWEATHERMAP_API_KEY"] = "tu-weather-api-key"

weather = OpenWeatherMapAPIWrapper()
weather_tool = Tool(
    name="Weather",
    description="Obtiene información del clima actual",
    func=weather.run
)

# 4.3 News API
from langchain_community.utilities import NewsAPIWrapper

os.environ["NEWS_API_KEY"] = "tu-news-api-key"

news = NewsAPIWrapper()
news_tool = Tool(
    name="News",
    description="Obtiene noticias actuales",
    func=news.run
)

# 4.4 Wikipedia
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(
        top_k_results=3,
        doc_content_chars_max=1000
    )
)

# 4.5 YouTube Search
from langchain_community.tools import YouTubeSearchTool

youtube_search = YouTubeSearchTool()

# ===============================================
# 5. HERRAMIENTAS DE CÁLCULO Y MATEMÁTICAS
# ===============================================

# 5.1 Calculator
from langchain_community.tools import LLMMathChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)
math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

calculator_tool = Tool(
    name="Calculator",
    description="Útil para cálculos matemáticos complejos",
    func=math_chain.run
)

# 5.2 WolframAlpha (requiere API key)
from langchain_community.utilities import WolframAlphaAPIWrapper

os.environ["WOLFRAM_ALPHA_APPID"] = "tu-wolfram-key"

wolfram = WolframAlphaAPIWrapper()
wolfram_tool = Tool(
    name="WolframAlpha",
    description="Resuelve problemas matemáticos y científicos complejos",
    func=wolfram.run
)

# 5.3 Python REPL
from langchain_experimental.tools import PythonREPLTool

python_repl = PythonREPLTool()

# ===============================================
# 6. HERRAMIENTAS DE EMAIL Y COMUNICACIÓN
# ===============================================

# 6.1 Gmail (requiere configuración OAuth)
from langchain_community.agent_toolkits import GmailToolkit

def crear_gmail_tools():
    """Requiere configuración OAuth2"""
    toolkit = GmailToolkit()
    return toolkit.get_tools()

# 6.2 Slack
from langchain_community.tools.slack.tool import SlackGetChannel

slack_tool = SlackGetChannel()

# ===============================================
# 7. HERRAMIENTAS PERSONALIZADAS
# ===============================================

# 7.1 Herramienta simple personalizada
def mi_funcion_personalizada(entrada: str) -> str:
    """Función personalizada de ejemplo"""
    return f"Procesado: {entrada.upper()}"

herramienta_personalizada = Tool(
    name="Mi_Herramienta",
    description="Convierte texto a mayúsculas",
    func=mi_funcion_personalizada
)

# 7.2 Herramienta estructurada personalizada
class EntradaBusqueda(BaseModel):
    """Entrada para herramienta de búsqueda"""
    query: str = Field(description="Término de búsqueda")
    num_results: int = Field(default=5, description="Número de resultados")

class HerramientaBusquedaPersonalizada(BaseTool):
    name = "busqueda_personalizada"
    description = "Búsqueda personalizada con parámetros específicos"
    args_schema = EntradaBusqueda
    
    def _run(
        self, 
        query: str, 
        num_results: int = 5,
        run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        # Lógica de búsqueda personalizada aquí
        return f"Buscando '{query}' con {num_results} resultados"
    
    async def _arun(
        self,
        query: str,
        num_results: int = 5,
        run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        # Versión asíncrona
        return f"Búsqueda asíncrona: '{query}'"

herramienta_estructurada = HerramientaBusquedaPersonalizada()

# ===============================================
# 8. HERRAMIENTAS DE CONVERSIÓN Y PROCESAMIENTO
# ===============================================

# 8.1 Text Processing Tools
def extraer_urls(texto: str) -> str:
    """Extrae URLs de un texto"""
    import re
    urls = re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', texto)
    return f"URLs encontradas: {', '.join(urls)}"

url_extractor = Tool(
    name="URL_Extractor",
    description="Extrae URLs de un texto",
    func=extraer_urls
)

# 8.2 JSON Processor
import json

def procesar_json(json_string: str) -> str:
    """Procesa y valida JSON"""
    try:
        data = json.loads(json_string)
        return f"JSON válido con {len(data)} elementos"
    except json.JSONDecodeError as e:
        return f"JSON inválido: {str(e)}"

json_processor = Tool(
    name="JSON_Processor",
    description="Procesa y valida strings JSON",
    func=procesar_json
)

# ===============================================
# 9. HERRAMIENTAS DE IMÁGENES Y MULTIMEDIA
# ===============================================

# 9.1 Image Analysis (requiere bibliotecas adicionales)
def analizar_imagen(ruta_imagen: str) -> str:
    """Analiza una imagen (requiere PIL y otros)"""
    try:
        from PIL import Image
        img = Image.open(ruta_imagen)
        return f"Imagen: {img.size[0]}x{img.size[1]} píxeles, modo: {img.mode}"
    except Exception as e:
        return f"Error al analizar imagen: {str(e)}"

image_analyzer = Tool(
    name="Image_Analyzer",
    description="Analiza propiedades básicas de imágenes",
    func=analizar_imagen
)

# ===============================================
# 10. TOOLKIT INTEGRADO - COMBINANDO HERRAMIENTAS
# ===============================================

class ToolkitCompleto:
    """Clase que organiza todas las herramientas disponibles"""
    
    def __init__(self):
        self.herramientas_busqueda = [
            ddg_search,
            wikipedia,
            youtube_search
        ]
        
        self.herramientas_archivos = [
            read_tool,
            write_tool,
            list_tool
        ]
        
        self.herramientas_calculo = [
            calculator_tool,
            python_repl
        ]
        
        self.herramientas_web = [
            requests_get,
            requests_post
        ]
        
        self.herramientas_personalizadas = [
            herramienta_personalizada,
            herramienta_estructurada,
            url_extractor,
            json_processor
        ]
    
    def obtener_todas_herramientas(self) -> List[Tool]:
        """Retorna todas las herramientas disponibles"""
        todas = []
        todas.extend(self.herramientas_busqueda)
        todas.extend(self.herramientas_archivos)
        todas.extend(self.herramientas_calculo)
        todas.extend(self.herramientas_web)
        todas.extend(self.herramientas_personalizadas)
        return todas
    
    def obtener_herramientas_por_categoria(self, categoria: str) -> List[Tool]:
        """Obtiene herramientas por categoría"""
        categorias = {
            'busqueda': self.herramientas_busqueda,
            'archivos': self.herramientas_archivos,
            'calculo': self.herramientas_calculo,
            'web': self.herramientas_web,
            'personalizada': self.herramientas_personalizadas
        }
        return categorias.get(categoria, [])
    
    def obtener_info_herramientas(self) -> Dict[str, Any]:
        """Información sobre todas las herramientas"""
        info = {}
        for herramienta in self.obtener_todas_herramientas():
            info[herramienta.name] = {
                'descripcion': herramienta.description,
                'tipo': type(herramienta).__name__
            }
        return info

# ===============================================
# 11. EJEMPLOS DE USO PRÁCTICO
# ===============================================

def ejemplo_uso_herramientas():
    """Ejemplos prácticos de uso de herramientas"""
    
    # Crear toolkit
    toolkit = ToolkitCompleto()
    
    # 1. Búsqueda web
    print("=== BÚSQUEDA WEB ===")
    resultado = ddg_search.run("Python programación 2024")
    print(f"Resultado búsqueda: {resultado[:200]}...")
    
    # 2. Calculadora
    print("\n=== CALCULADORA ===")
    calculo = calculator_tool.run("¿Cuánto es 15 * 23 + 45?")
    print(f"Resultado cálculo: {calculo}")
    
    # 3. Procesamiento de archivos
    print("\n=== ARCHIVOS ===")
    try:
        archivos = list_tool.run("./")
        print(f"Archivos en directorio: {archivos}")
    except Exception as e:
        print(f"Error listando archivos: {e}")
    
    # 4. Herramienta personalizada
    print("\n=== PERSONALIZADA ===")
    resultado_personal = herramienta_personalizada.run("hola mundo")
    print(f"Resultado personalizado: {resultado_personal}")
    
    # 5. Información del toolkit
    print("\n=== INFO TOOLKIT ===")
    info = toolkit.obtener_info_herramientas()
    for nombre, detalles in list(info.items())[:3]:  # Solo primeras 3
        print(f"{nombre}: {detalles['descripcion']}")

# ===============================================
# 12. CONFIGURACIÓN Y MEJORES PRÁCTICAS
# ===============================================

def configurar_herramientas_seguras():
    """Configuraciones recomendadas para herramientas"""
    
    # Limitar herramientas de archivo a directorios específicos
    herramientas_seguras = [
        ReadFileTool(base_dir="./safe_documents"),
        WriteFileTool(base_dir="./safe_output"),
        ListDirectoryTool(base_dir="./safe_documents")
    ]
    
    # Configurar timeouts para herramientas web
    requests_tool_seguro = RequestsGetTool(
        allowed_schemas=["https"],
        requests_wrapper=None  # Aquí puedes añadir configuración de timeout
    )
    
    return herramientas_seguras + [requests_tool_seguro]

def crear_herramienta_con_validacion(func, nombre: str, descripcion: str):
    """Wrapper para añadir validación a herramientas"""
    
    def wrapper_validado(entrada: str) -> str:
        if not entrada or len(entrada.strip()) == 0:
            return "Error: Entrada vacía"
        
        try:
            return func(entrada)
        except Exception as e:
            return f"Error en {nombre}: {str(e)}"
    
    return Tool(
        name=nombre,
        description=descripcion,
        func=wrapper_validado
    )

# ===============================================
# 13. HERRAMIENTAS AVANZADAS Y EXPERIMENTALES
# ===============================================

# Shell Tool (usar con precaución)
from langchain_experimental.tools import ShellTool

shell_tool = ShellTool()  # ¡PELIGROSO! Solo para desarrollo

# Human Input Tool
from langchain_community.tools import HumanInputRun

human_input = HumanInputRun(
    name="Human_Input",
    description="Solicita input del usuario cuando sea necesario"
)

# ===============================================
# FUNCIÓN PRINCIPAL DE DEMOSTRACIÓN
# ===============================================

if __name__ == "__main__":
    print("🔧 DEMO: Herramientas LangChain")
    print("=" * 50)
    
    # Ejecutar ejemplos
    ejemplo_uso_herramientas()
    
    # Mostrar toolkit completo
    toolkit = ToolkitCompleto()
    todas_herramientas = toolkit.obtener_todas_herramientas()
    
    print(f"\n📊 Total de herramientas disponibles: {len(todas_herramientas)}")
    print("\n🏷️  Herramientas por categoría:")
    categorias = ['busqueda', 'archivos', 'calculo', 'web', 'personalizada']
    for cat in categorias:
        herrs = toolkit.obtener_herramientas_por_categoria(cat)
        print(f"  - {cat.capitalize()}: {len(herrs)} herramientas")
    
    print("\n✅ Demo completada!")

# ===============================================
# INSTALACIÓN REQUERIDA
# ===============================================

"""
Para usar todas estas herramientas, necesitas instalar:

pip install langchain langchain-community langchain-openai
pip install duckduckgo-search wikipedia-api youtube-search-python
pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib
pip install wolframalpha newsapi-python pyowm
pip install faiss-cpu  # o faiss-gpu
pip install pandas openpyxl  # para CSV/Excel
pip install PyPDF2 pypdf  # para PDFs
pip install pillow  # para imágenes
pip install requests beautifulsoup4  # para web scraping

# Herramientas experimentales (opcional)
pip install langchain-experimental

# Para bases de datos
pip install sqlalchemy pymongo redis

# Para herramientas de Gmail/Slack
pip install google-auth google-auth-oauthlib google-auth-httplib2
pip install slack-sdk
"""